# CSCI 3155: Assignment 9

Topics: 
- Basics of Objects
- Traits

Readings: Notes posted on canvas.

__Name__: Connor O'Reilly

In [1]:
// TEST HELPER
def passed(points: Int) {
    require(points >=0)
    if (points == 1) print(s"\n*** Tests Passed (1 point) ***\n")
    else print(s"\n*** Tests Passed ($points points) ***\n")
}

defined function passed

## Problem 1 (30 points): Objects in Scala

### A (12 points)
Consider a inventory tracking application for a store like Home Depot. Below is an `Item` class with one field called `name` which is a string.  
1) Fill in the definition of its `equals` method such that it returns true if and only if its argument is an item with the same name. 

Next, we define a `Product` class that has three fields: `name`, `skn` (stock-keeping number), `qty` (quantity in stock), and `price`.

1) As given, scala will complain that class Product cannot extend the abstract class Item. Add the missing methods to enable the class Product to properly extend Item

2) Write a factory pattern that takes in a formatted string of the form "Item Name, SKN, Qty, Price" and builds a Product object with these attributes. 
    - Item name is a string with possible white spaces, e.g., " Lint Roller "
    - SKN is an alpha-numeric string, .e.g., "1a98hn6", 
    - Qty is an integer, e.g., 98.
    - Price is a Double, e.g., 23.35.
    
   You may want to use `scala.util.matching.Regex` [https://www.scala-lang.org/api/2.12.5/scala/util/matching/Regex.html] If the input fails to match the pattern, an `IllegalArgumentException` must be thrown

Companion objects are covered in scala book Chapter 4.3. Also lookup here: https://alvinalexander.com/scala/factory-pattern-in-scala-design-patterns

In [2]:
import scala.util.matching.Regex

abstract class Item(val name: String) {
    /* Needs a method to return a string describing the product */
    def toString: String
    /* Two products are equal if their SKNs are equal */
    def equals (p: Item): Boolean = {
        name == p.name     
    }
}

/* 
  Note that for a product to be equal to an object of type Item, that object
  must be a product with matching skn 
*/

class Product(override val name: String, 
              val skn: String, 
              val qty: Int,
              val price: Double) extends Item(name) {
    override def toString: String = name
    def equals(p: Product): Boolean = {
        skn == p.skn
    }
}


object Product {
    def apply(formattedInput: String): Product = {
        /* 
        Input format must be 
            (name with possible spaces)[optional whitespaces],[optional whitespaces](SKN)[optional whitespaces],[optional whitespaces](quantity)[optional whitespaces],[optional whitespaces](price)
        */
        //BEGIN SOLUTION
        val pattern = """(^[A-Za-z]\w+\s*\w+)\s*,\s*([A-Za-z0-9]\w+)\s*,\s*([0-9]\w+)\s*,\s*([0-9]\w+.*[0-9])\s*""".r
        formattedInput match{
            case pattern(name,skn,qty,price) => {
                new Product(name,skn,qty.toInt,price.toDouble)
            }
            case _ => {
                throw new IllegalArgumentException("Unhandled case")
            }
        }
        //END SOLUTION
    }
}

import scala.util.matching.Regex


defined class Item
defined class Product
defined object Product

In [3]:
class MyItem(override val name: String) extends Item(name){
        override def toString: String = {
        name
    }
}
val p1 = new MyItem("Eggs")
assert(p1.equals(new MyItem("Eggs")), "Failed")
assert(!p1.equals(new MyItem("Not Eggs")), "Failed")
passed(3)


*** Tests Passed (3 points) ***


defined class MyItem
p1: MyItem = Eggs

In [4]:
//BEGIN TEST
val p1 = Product("Baby Wipes, 1a2e34, 100, 22.03")
println(p1.toString)
assert(p1.name == "Baby Wipes", "Name not parsed correctly")
assert(p1.skn == "1a2e34", "SKN not parsed correctly")
assert(p1.qty == 100, "qty not parsed correctly")
assert(p1.equals(new Product("Infant Wet Wipes", "1a2e34", 9, 22.04)))
passed(3)
//END TEST

Baby Wipes

*** Tests Passed (3 points) ***


p1: Product = Baby Wipes

In [5]:
//BEGIN TEST
try {
    val p2 = Product("Badly Formatted String, 2bce54, $3102, 45.89")
    assert(false, "Failed, the string is badly formatted but your code is OK with it.")
} catch {
    case e => println(s"Expected behavior seen")
}
passed(3)
//END TEST

Expected behavior seen

*** Tests Passed (3 points) ***


In [6]:
//BEGIN TEST
val e1 = Product("Wiper Blades    , 99a2ef2   , 119,  100.00")
println(e1.toString)
assert(e1.name == "Wiper Blades", "Product name not parsed correctly")
assert(e1.skn == "99a2ef2", "SKN not parsed correctly")
assert(e1.qty == 119, "Qty not parsed correctly")
assert(!e1.equals(new Product("Wiper Blades", "random", 119, 100.00)))
passed(3)
//END TEST

Wiper Blades

*** Tests Passed (3 points) ***


e1: Product = Wiper Blades

### B (8 points)
A store like Home Depot sells services too, .e.g., Door installation. We therefore wish to add a new subclass of `Item` to our system. The subclass must be named "Service" with the information associated should involve their name (string), service_id (string) and rate (double).

Write a new class named `Service` whose constructor should take in the name of the service (String), its serviceId (string), and its rate (i.e., its price per hour) which is a Double. It has to extend `Item` class. 

A Service object is equal to an object p of type Item, only if p is actually an instance of Service, and if their names are the same and they have the same serviceId number.

In [7]:
//BEGIN SOLUTION
class Service(override val name: String, val service_id: String, val rate: Double) extends Item(name)
{
    override def toString: String = name
    override def equals(i: Item): Boolean = {
        if(i.isInstanceOf[Service]){
            val s: Service =  i.asInstanceOf[Service]
            s.name == name && s.service_id == service_id
        }else{
            false
        }
    }
}
//END SOLUTION

defined class Service

In [8]:
//BEGIN TEST
val e1 = Product("Barn Door, 2014ae7, 10, 568.09 ")
val c1 = new Service("Door installation", "20221a98", 35.00)
assert( !c1.equals(e1), "Failed")
assert( c1.equals(new Service("Door installation", "20221a98", 45.00)), "Failed")
passed(4)
//END TEST


*** Tests Passed (4 points) ***


e1: Product = Barn Door
c1: Service = Door installation

In [9]:
//BEGIN TEST
val s1 = new Service("Door installation", "20221a98", 35.00)
val s2 = new Service("Barn Door installation", "20221a98", 35.00)
assert( !s1.equals(s2), "Failed")
passed(4)
//END TEST


*** Tests Passed (4 points) ***


s1: Service = Door installation
s2: Service = Barn Door installation

### B (10 points)
Write a function `computeCost` that takes an object of type `Item` and an Integer `nos` denoting the number of such items, and computes the total cost. `computeCost` must behave differently for different `Item` objects:
* If the `Item` object is an instance of `Product`, then the total cost is the product of `price` and `nos`. 
* If the `Item` object is an instance `Service`, then the total cost is the product of `rate` and `nos`.
* Else throw `IllegalArgumentException`

In [10]:
def computeCost(i:Item, nos:Int): Double = {
    //BEGIN SOLUTION
    if(i.isInstanceOf[Product]){
        val p: Product =  i.asInstanceOf[Product]
        p.price * nos
    }
    else if(i.isInstanceOf[Service]){
        val s: Service = i.asInstanceOf[Service]
        s.rate * nos
    }
    else{
        throw new IllegalArgumentException("unhandled case")
    }
    //END SOLUTION
}

defined function computeCost

In [11]:
assert(computeCost(Product("Barn Door, 2014ae7, 10, 568.09 "),2) == 1136.18, "Failed")
assert(computeCost(Product("Barn Door, 2014ae7, 10, 568.09 "),2) == 
       computeCost(Product("Washing Machine, 9024ae7, 11, 1136.18 "),1), "Failed")
assert(computeCost(new Service("Door installation", "20221a98", 35.00), 10) == 350.0, "Failed")
class MyItem(override val name: String) extends Item(name){
        override def toString: String = {
        name
    }
}
try {
    val p2 = new MyItem("generic")
    computeCost(p2, 10)
    assert(false, "Failed, Your code computes the cost of an item without knowing its price or rate")
} catch {
    case e => println(s"Expected behavior seen")
}
passed(10)

Expected behavior seen

*** Tests Passed (10 points) ***


defined class MyItem

## Problem 2 (20 points)

Suppose we wish to define an abstract class called `Shape` that is a three-dimensional geometric shape. It needs to have methods such as  `getCenter`,  and `translate`.

  - `getCenter` : Gets the center of the shape. For a polyhedron the center is the "average" of the coordinates of its vertices. For a sphere/ellipsoid, the center is part of the description of the object (see below).
  - `translate`: Translates the shape along some given values of `xShift`, `yShift`, and `zShift`.


We will define a trait called `WithCorners` that define shapes with corners. This trait will implement a function called `getVertices`.

There are many classes that inherit from `Shape` including `Polyhedron` and `Ellipsoid`. The classes `Pyramid` and `Cuboid` inherit from `Polyhedron` and `Sphere` inherits from `Ellipsoid`. 

Also the class `Polyhedron` and all its derived classes must mixin the trait `WithCorners` but the class `Ellipsoid` and its derived classes should not.

- class `Pyramid(x0: (Double, Double, Double), x1: (Double, Double, Double), x2: (Double, Double, Double), x3: (Double, Double, Double))` must have four class parameters representing the four corners of the pyramid.
  - It must inherit from `Polyhedron`.
  - It must mixin the trait `WithCorners`.
  - It must override the `translate` method so that `translate` called on a `Pyramid` returns an instance of a `Pyramid`.
  
- class `Cuboid(lowerLeft: (Double, Double, Double), length: Double, width: Double, height:Double)` must have the class parameters representing the lower left coordinate, length,  width, and height.
  - It must inherit from `Polyheron`.
  - It must mixin the trait `WithCorners`.
  - It must override the `translate` method so that `translate` called on a `Cuboid` returns an instance of a `Cuboid`.
  
- class `Sphere(center: (Double, Double, Double), rad: Double)` must have its center coordinate and radius as class paramters.
  - It must inherit from Ellipsoid.
  - It must override the `translate` methodd so that `translate` called on a `Sphere` returns an instance of a `Sphere`.





### Part A
Complete the definitions below so that your code compiles.

**Restrictions** No loops, mutables (var) or recursion. Please use functors `map`, `foldLeft` etc.. when possible.


In [32]:
abstract class Shape {
    def getCenter:(Double, Double, Double)
    def translate(xShift: Double, yShift: Double, zShift: Double): Shape
}

trait WithCorners {
    def getVertices: List[(Double, Double, Double)]
}

class Ellipsoid(val center: (Double, Double, Double), val axisLengths: (Double, Double, Double)) extends Shape {
    //TODO: Finish the methods that need to be implemented.
    //BEGIN SOLUTION
    override def getCenter: (Double, Double, Double) = center
    override def translate(xShift: Double, yShift: Double, zShift: Double): Ellipsoid ={
        val cent_old = getCenter
        val cent_new = (cent_old._1+xShift, cent_old._2+yShift, cent_old._3+zShift)
        new Ellipsoid(cent_new, axisLengths)
    }
    //END SOLUTION 
}

class Polyhedron(val listOfVerts: List[(Double, Double, Double)]) extends Shape with WithCorners {
     assert(listOfVerts.length >= 1)
    // TODO: Finish the methods that need to be implemented.
    //BEGIN SOLUTION
    
    override def getVertices:List[(Double, Double, Double)] = listOfVerts //just return list of vertices
    
    //translate
    override def translate(xShift: Double, yShift: Double, zShift: Double): Polyhedron = {
        val new_lst_verts: List[(Double, Double, Double)] = listOfVerts.map( (x: (Double, Double, Double)) => {
            (x._1 + xShift, x._2 + yShift, x._3 + zShift)
        })
        new Polyhedron(listOfVerts)
    }
    def getCenter: (Double,Double,Double) = {
        //get length of list
        val len = listOfVerts.length
        //get sum
        val sum = listOfVerts.foldLeft[(Double, Double, Double)]((0,0,0))((acc:(Double, Double, Double), elt:(Double, Double, Double)) => {
            ( ((acc._1 + elt._1)), ((acc._2 + elt._2)), ((acc._3 + elt._3)) )
        })
        //get average
        (sum._1/len, sum._2/len,sum._3/len)
    }
    //END SOLUTION
}

//TODO: Complete definitions of triangle, rectangle and circle classes.
//BEGIN SOLUTION
class Pyramid(val x0: (Double, Double, Double), val x1: (Double, Double, Double), val x2: (Double, Double, Double), val x3: (Double, Double, Double)) extends Polyhedron(List[(Double,Double,Double)](x0,x1,x2,x3)) {
    override def translate(xShift: Double, yShift: Double, zShift: Double): Pyramid = {
        val verts : List[(Double, Double, Double)] = List(x0,x1,x2,x3)
        val new_lst_verts: List[(Double, Double, Double)] = verts.map( (x: (Double, Double, Double)) => {
            (x._1 + xShift, x._2 + yShift, x._3 + zShift)
        })
        new Pyramid(new_lst_verts(0),new_lst_verts(1),new_lst_verts(2),new_lst_verts(3))
    }
    
}

class Cuboid(val lowerLeft: (Double, Double, Double), val length: Double, val width: Double, val height:Double) extends Polyhedron(List[(Double,Double,Double)]
                                                                                                                   (lowerLeft, (lowerLeft._1, lowerLeft._2 + width,lowerLeft._3), (lowerLeft._1, lowerLeft._2, lowerLeft._3 + height),(lowerLeft._1, lowerLeft._2 + width, lowerLeft._3 + height),(lowerLeft._1 + length, lowerLeft._2,lowerLeft._3), (lowerLeft._1 + length, lowerLeft._2 + width,lowerLeft._3), (lowerLeft._1 + length, lowerLeft._2, lowerLeft._3 + height),(lowerLeft._1 + length, lowerLeft._2 + width, lowerLeft._3 + height) ))
                                                                                                                   {
        
        //tf are the cuboid vertices
        //List(lowerLeft, (lowerLeft._1, lowerleft._2 + width,lowerLeft._3), (lowerLeft._1, lowerleft._2, lowerLeft._3 + height),(lowerLeft._1, lowerleft._2 + width, lowerLeft._3 + height),(lowerLeft._1 + length, lowerleft._2,lowerLeft._3), (lowerLeft._1 + length, lowerleft._2 + width,lowerLeft._3), (lowerLeft._1 + length, lowerleft._2, lowerLeft._3 + height),(lowerLeft._1 + length, lowerleft._2 + width, lowerLeft._3 + height) )
        override def translate(xShift: Double, yShift: Double, zShift: Double): Cuboid = {
            val new_lowerLeft: (Double, Double, Double) = (lowerLeft._1 + xShift, lowerLeft._2 + yShift, lowerLeft._3 + zShift)
            new Cuboid(new_lowerLeft, length, width,height)
        }
}

class Sphere( center: (Double, Double, Double), val rad: Double) extends Ellipsoid(center, (rad,rad,rad) ) {

    override def translate(xShift: Double, yShift: Double, zShift: Double): Sphere ={
        val cent_old = getCenter
        val cent_new = (cent_old._1+xShift, cent_old._2+yShift, cent_old._3+zShift)
        new Sphere(cent_new, rad)
    }
}

//END SOLUTION

defined class Shape
defined trait WithCorners
defined class Ellipsoid
defined class Polyhedron
defined class Pyramid
defined class Cuboid
defined class Sphere

In [33]:
val el1 = new Ellipsoid((5.0, -3.5, 0.0), (2.1, 1.3, 2.0))
val el2 = el1.translate(3,4,3)
assert (el2.center == (8.0, 0.5, 3.0), "Test failed: After translation, the center must be (8,0.5,3.0)")

val pl1 = new Polyhedron(List((0,0,0), (2,1,2)))
assert (pl1.getCenter == (1.0, 0.5, 1.0), "center must be (1,0.5)")

passed(5)


*** Tests Passed (5 points) ***


el1: Ellipsoid = ammonite.$sess.cmd31$Helper$Ellipsoid@4c5e253b
el2: Ellipsoid = ammonite.$sess.cmd31$Helper$Ellipsoid@1079a3c4
pl1: Polyhedron = ammonite.$sess.cmd31$Helper$Polyhedron@7641606a

In [34]:
val pyr = new Pyramid( (0,0, 0), (2,0,0), (1,1,0), (1.5, 1.5, 3))
val pyr2 = pyr.translate(1,1,1)
assert (pyr2.isInstanceOf[Pyramid])
print(pyr2.getCenter)
assert(pyr2.getCenter._1 == 2.125)
assert(pyr2.getCenter._3 == 1.75)
passed(5)

(2.125,1.625,1.75)
*** Tests Passed (5 points) ***


pyr: Pyramid = ammonite.$sess.cmd31$Helper$Pyramid@289bbaf9
pyr2: Pyramid = ammonite.$sess.cmd31$Helper$Pyramid@2e186142

In [35]:
val cub = new Cuboid((-1,-1,-1), 2, 2, 2)
val listOfVerts = cub.getVertices
assert(listOfVerts.length == 8)
print(cub.getCenter)
assert (cub.getCenter == (0,0,0))
val cub2: Cuboid = cub.translate(1,1,1)
print(cub2.getCenter)
assert(cub2.getCenter == (1,1,1))
passed(5)

(0.0,0.0,0.0)(1.0,1.0,1.0)
*** Tests Passed (5 points) ***


cub: Cuboid = ammonite.$sess.cmd31$Helper$Cuboid@2123b73d
listOfVerts: List[(Double, Double, Double)] = List(
  (-1.0, -1.0, -1.0),
  (-1.0, 1.0, -1.0),
  (-1.0, -1.0, 1.0),
  (-1.0, 1.0, 1.0),
  (1.0, -1.0, -1.0),
  (1.0, 1.0, -1.0),
  (1.0, -1.0, 1.0),
  (1.0, 1.0, 1.0)
)
cub2: Cuboid = ammonite.$sess.cmd31$Helper$Cuboid@7c991b15

In [36]:
val spr = new Sphere((1,1,1), 3)
val new_spr: Sphere = spr.translate(2,2,2)
assert (new_spr.center == (3,3,3))
assert (new_spr.rad == 3)
passed(5)


*** Tests Passed (5 points) ***


spr: Sphere = ammonite.$sess.cmd31$Helper$Sphere@13e47b82
new_spr: Sphere = ammonite.$sess.cmd31$Helper$Sphere@1a0b9ef0